In [3]:
import requests
import pandas as pd
import os

In [4]:
# change if necessary
port = 8983
core_name = "new_core"
main_data_dir = ".\Data"

In [59]:
url = "http://localhost:8983/solr/new_core/suggest?suggest=true&suggest.dictionary=mySuggester&suggest.q=Prejudice"
response = requests.get(url)
print(response.json())

{'responseHeader': {'status': 0, 'QTime': 17}, 'suggest': {'mySuggester': {'Prejudice': {'numFound': 0, 'suggestions': []}}}}


In [5]:
def ClearAllDocuments():
    url = "http://localhost:" + str(port) + "/solr/" + core_name + "/update?stream.body=<delete><query>*:*</query></delete>&commit=true"
    response = requests.get(url)
    print(response)

# Uploads all files in the directory that you pointed to
def UploadDirectory(dir):
    os.system("java -jar -Dc=new_core -Dauto solr-9.2.1\example\exampledocs\post.jar " + dir)
    return

In [6]:
ClearAllDocuments()

<Response [200]>


In [16]:
UploadDirectory(main_data_dir+"\*")

In [11]:
# experimental function
def GetStats():
    url = "http://localhost:8983/solr/new_core/select?fq=TYPE%3APOST&indent=true&q.op=OR&q=category%3ARomance&useParams=&stats=true&stats.field=upvotes"
    response = requests.get(url)
    print(response.json())

In [12]:
GetStats()

{'responseHeader': {'status': 0, 'QTime': 2, 'params': {'q': 'category:Romance', 'indent': 'true', 'stats': 'true', 'q.op': 'OR', 'fq': 'TYPE:POST', 'useParams': '', 'stats.field': 'upvotes'}}, 'response': {'numFound': 1957, 'start': 0, 'numFoundExact': True, 'docs': [{'id': '14vrpiu', 'title': ['Pride and Prejudice by you already know who'], 'selftext': "SPOILERS!! THIS IS YOUR LAST WARNING!!\n\nWhen I tell you I am so glad to be freed from the shackles of this book. I enjoyed it, but it was so long to read I can't help but be glad it's over.\n\nI doubt I have much to be said on the book that hadn't already been said, so I think I'll give my opinions on the characters instead:\n\n> The Bennets\n\nMrs Bennet - Literally unhinged\n\nMr Bennet - Unbothered but probably should be\n\nJane - The least problematic person in this god forsaken family. Gossips with Lizzie about literally everything. I like how close the two are.\n\nLizzie - In her hot girl summer era throughout the entire book.

In [11]:
book_data = pd.read_csv("./Data/book_data_old.csv")
book_data

,id,book_title,book_author,publication_date,description,cover_path,category
0,hj1d1_10,Pride and Prejudice,Jane Austen,1813-01-28T00:00:00Z,Pride and Prejudice follows the turbulent rela...,./Book_Covers/pride_and_prejudice,Romance
1,hj1d1_11,Frankenstein,Mary Shelley,1818-01-01T00:00:00Z,Frankenstein tells the story of gifted scienti...,./Book_Covers/frankenstein,Horror


In [12]:
book_data.to_csv("book_data_processed.csv",index=False)

## Testing timeline search

In [3]:
url = "http://localhost:8983/solr/new_core/select?indent=true&q.op=OR&q=book%3A%22Harry%20Potter%22&fq=created_utc:[2022-01-01T00:00:00Z TO 2022-12-31T23:59:59Z]"
response = requests.get(url)
print(response.json())

{'responseHeader': {'status': 0, 'QTime': 26, 'params': {'q': 'book:"Harry Potter"', 'indent': 'true', 'q.op': 'OR', 'fq': 'created_utc:[2022-01-01T00:00:00Z TO 2022-12-31T23:59:59Z]'}}, 'response': {'numFound': 560, 'start': 0, 'numFoundExact': True, 'docs': [{'id': 'ryeord_1', 'post_id': 'ryeord', 'comment_text': 'I remember there being a huge controversy about the casting of Cho Chang in the movies, with some being upset that the actress had a Scottish accent while others were upset that she was of East Asian descent in the first place.\r\n\r\nMy only thought about the matter: what the fuck did you expect? The movies take place in the UK, and JKR was adamant that every character be from either the UK or Ireland. And were the people upset that she is East Asian in appearance also upset that the Patil twins are Indian in appearance?\r\n\r\nOf all the Harry Potter controversies, this was easily the most baffling.', 'author': 'jah05r', 'author_s': 'jah05r', 'created_utc': '2022-01-07T21